https://colab.research.google.com/drive/1TlIfecAt51WsskxSknGm_WOMbA6h9tF4

In [1]:
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz

--2019-10-01 06:23:56--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘UrbanSound8K.tar.gz’

UrbanSound8K.tar.gz 100%[===================>]   5.61G  56.6MB/s    in 89s     

2019-10-01 06:25:26 (64.5 MB/s) - ‘UrbanSound8K.tar.gz’ saved [6023741708/6023741708]



In [2]:
!ls

sample_data  UrbanSound8K.tar.gz


In [0]:
!tar xzvf UrbanSound8K.tar.gz

In [0]:
!pwd

/content


In [0]:
import pandas as pd
data = pd.read_csv('/content/UrbanSound8K/metadata/UrbanSound8K.csv') # Refer to the location of downloaded file here

In [0]:
import librosa
import numpy as np

In [0]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return np.hstack([mfccs,chroma,mel,contrast,tonnetz])

In [0]:
from glob import glob
import numpy as np
filenames = np.array(glob('/content/UrbanSound8K/audio/*/*.wav'))

In [8]:
len(filenames)

8732

In [9]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [0]:
ids = data['slice_file_name'].values
folds = data['fold'].values

In [0]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    return mfccs

In [12]:
i=0
'/content/UrbanSound8K/audio/fold'+str(folds[i])+'/'+str(ids[i])

'/content/UrbanSound8K/audio/fold5/100032-3-0-0.wav'

In [0]:
x = []
y = []
for i in range(len(ids)):
  if(i%100==0):
    print(i)
  try:
    filename = '/content/UrbanSound8K/audio/fold'+str(folds[i])+'/'+str(ids[i])
    y.append(data[data['slice_file_name']==ids[i]]['class'].values)
    x.append(extract_feature(filename))
  except:
    continue
  

In [0]:
np.array(x).shape

(8732, 40)

In [0]:
y2 = []
for i in range(len(y)):
  y2.append(y[i][0])

In [0]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
y3 = np.array(pd.get_dummies(y2))

In [0]:
x = np.array(x)

In [0]:
print(x.shape, y3.shape)

(8732, 40) (8732, 10)


In [0]:
x = x.reshape(x.shape[0],x.shape[1],1)

In [0]:
x = x.reshape(x.shape[0],x.shape[1])

In [0]:
import os
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.layers import Embedding
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Bidirectional

In [0]:
model = Sequential()
model.add(Dense(1000, input_shape = (40,), activation = 'relu'))
#model.add(Dense(1000,activation='relu'))
model.add(Dense(10,activation='sigmoid'))
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              41000     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 51,010
Trainable params: 51,010
Non-trainable params: 0
_________________________________________________________________


In [0]:
# model = Sequential()
# model.add(Dense(1000, input_shape = (193,1), activation = 'relu'))
# model.add((LSTM(100)))
# model.add(Dense(1000,activation='relu'))
# model.add(Dense(10,activation='sigmoid'))
# model.summary()

In [0]:
from keras.optimizers import Adam
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

In [0]:
np.max(x)

269.6517343431436

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y3, test_size=0.30,random_state=10)

In [0]:
model.fit(X_train/np.max(x), y_train, epochs=100, batch_size=32, validation_data=(X_test/np.max(x), y_test), verbose = 1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 6112 samples, validate on 2620 samples
Epoch 1/100
6112/6112 [==============================] - 6s 979us/step - loss: 2.2672 - acc: 0.1702 - val_loss: 2.2356 - val_acc: 0.2046
Epoch 2/100
6112/6112 [==============================] - 1s 155us/step - loss: 2.1977 - acc: 0.2549 - val_loss: 2.1633 - val_acc: 0.2973
Epoch 3/100
6112/6112 [==============================] - 1s 157us/step - loss: 2.1174 - acc: 0.3262 - val_loss: 2.0715 - val_acc: 0.3710
Epoch 4/100
6112/6112 [==============================] - 1s 170us/step - loss: 2.0190 - acc: 0.3796 - val_loss: 1.9652 - val_acc: 0.3920
Epoch 5/100
6112/6112 [==============================] - 1s 156us/step - loss: 1.9139 - acc: 0.3974 - val_loss: 1.8635 - val_acc: 0.4221
Epoch 6/100
6112/6112 [==============================] - 1s 159us/step - loss: 1.8201 - acc: 0.4249 - val_loss: 1.7780 - val_acc: 0.4427
Epoch 7/100
6112/6112 [============